# Summary

This notebook explores [pipecat](https://github.com/pipecat-ai/pipecat), an open-source Python framework for building real-time voice and multimodal conversational agents.

In [2]:
import asyncio

import aiohttp
from loguru import logger
from pipecat.frames.frames import EndFrame, TextFrame, TTSSpeakFrame
from pipecat.pipeline.pipeline import Pipeline
from pipecat.pipeline.runner import PipelineRunner
from pipecat.pipeline.task import PipelineTask
from pipecat.services.google.tts import GoogleTTSService
from pipecat.services.piper.tts import PiperTTSService
from pipecat.transports.local.audio import (
    LocalAudioTransport,
    LocalAudioTransportParams,
)

In [ ]:
# Transport receives audio from the user (browser, phone, etc.)
# and streams audio back to the user
transport = LocalAudioTransport(
    params=LocalAudioTransportParams(audio_out_enabled=True)
)

tts = GoogleTTSService()

pipeline = Pipeline([tts, transport.output()])
task = PipelineTask(pipeline)

await task.queue_frames(
    [
        TTSSpeakFrame("Hello! This is Pipecat speaking from your Raspberry Pi."),
        # Adding the EndFrame to the queue makes it stop after speaking
        EndFrame(),
    ]
)

runner = PipelineRunner()
await runner.run(task)

2025-08-02 15:09:17.403 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineSource#4 -> GoogleTTSService#4
2025-08-02 15:09:17.409 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking GoogleTTSService#4 -> PipelineSink#4
2025-08-02 15:09:17.417 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineTaskSource#4 -> Pipeline#4
2025-08-02 15:09:17.422 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking Pipeline#4 -> PipelineTaskSink#4
2025-08-02 15:09:17.437 | DEBUG    | pipecat.pipeline.runner:run:71 - Runner PipelineRunner#4 started running PipelineTask#4
2025-08-02 15:09:17.456 | DEBUG    | pipecat.services.google.tts:run_tts:585 - GoogleTTSService#4: Generating TTS [Hello! This is Pipecat speaking from your Raspberry Pi.]
2025-08-02 15:09:19.013 | DEBUG    | pipecat.pipeline.runner:run:88 - Runner PipelineRunner#4 finished running PipelineTask#4


In [ ]:
transport = LocalAudioTransport(
    params=LocalAudioTransportParams(audio_out_enabled=True)
)

async with aiohttp.ClientSession() as session:
    # You need to download the model and start a Piper TTS server first:
    # uv run python3 -m piper.download_voices en_US-lessac-medium
    # uv run python3 -m piper.http_server -m en_US-lessac-medium
    # https://github.com/pipecat-ai/pipecat/pull/2332
    tts = PiperTTSService(
        base_url="http://127.0.0.1:5000",
        aiohttp_session=session,
        sample_rate=24000,
    )

    task = PipelineTask(Pipeline([tts, transport.output()]))

    await task.queue_frames(
        [
            TTSSpeakFrame("Hello! This is Pipecat speaking from your Raspberry Pi."),
            EndFrame(),
        ]
    )

    runner = PipelineRunner()
    await runner.run(task)

2025-08-02 15:14:00.084 | INFO     | pipecat:<module>:14 - ᓚᘏᗢ Pipecat 0.0.77 (Python 3.12.8 (main, Jan  8 2025, 19:09:30) [GCC 10.2.1 20210110]) ᓚᘏᗢ
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.bcm2835_headpho.pcm.front.0:CARD=0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5233:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM front
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.bcm2835_headpho.pcm.surround51.0:CARD=0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5233:(snd_config_expand